# Note: This notebook is largely ignorable. Have since deviated from this approach and not bothered to tidy the notebook significantly.

# Daily + weekly + yearly seasonality model

Modelling both daily and weekly seasonality with gaussian process on $log(lambda)$. Modelling yearly seasonality as a scale factor on $lambda$, centered around 1.

Next step after this is to include weather effects.

In [ ]:
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

import polars as pl
import numpy as np
import scipy

import pymc as pm
import arviz as az

import warnings

sns.set(rc={'figure.figsize':(17,11)})

warnings.filterwarnings("ignore")

## Load data

In [ ]:
data = pl.read_parquet("../data/counter_data.parquet").with_columns(pl.col("weekday") - 1)

In [ ]:
filtered_data = data.filter(
    (pl.col("year") == 2022) &
    (pl.col("site_name") == "Thorndon Quay")
).with_columns(
    pl.col("count_outgoing").fill_null(0),
    (
        (
            pl.col("record_time").dt.date() - date(year=2022, month=1, day=1)
        ).dt.days()
    ).alias("day_in_year")
)

## Visualise observations

In [ ]:
sns.relplot(
    data.filter(
        (pl.col("hour") >= 7) & 
        (pl.col("hour") <= 9) &
        (pl.col("weekday") < 5)
    ).group_by(
        "day_in_year"
    ).agg(
        pl.col("count_outgoing").mean().alias("mean_count")
    ),
    x="day_in_year",
    y="mean_count"
)

In [ ]:
sns.relplot(
    filtered_data.filter(
        (pl.col("hour") >= 7) & 
        (pl.col("hour") <= 9) &
        (pl.col("weekday") < 5)
    ).set_sorted(
        "record_time"
    ).group_by_dynamic(
        "record_time",
        every="1mo"
    ).agg(
        pl.col("count_outgoing").sum().alias("month_count")
    ),
    x="record_time",
    y="month_count"
)
plt.ylim(0,7000)
plt.title("Total count for each month")

In [ ]:
sns.relplot(
    (
        filtered_data.filter(
            (pl.col("day") > 7) & (pl.col("day") <= 14)
        ).with_columns(
            (pl.col("weekday") * 24 + pl.col("hour")).alias("day_hour")
        )
    ),
    x="day_hour", 
    y="count_outgoing",
    kind="line",
    col="month",
    col_wrap=3,
)

## Model

In [ ]:
with pm.Model() as m_4:
    hour = pm.MutableData("hour", np.array(filtered_data["hour"]), dims="obs_id")
    weekday = pm.MutableData("weekday", np.array(filtered_data["weekday"]), dims="obs_id")
    month = pm.MutableData("month", np.array(filtered_data["month"]) - 1, dims="obs_id")
    
    ls = pm.Exponential("ls", scale=[2, 1, 24])
    
    # hourly periodic effect
    cov_daily = pm.gp.cov.Periodic(1, period=24, ls=ls[0])
    
    # allowing variability to 24-hour period
    cov_daily *= pm.gp.cov.ExpQuad(1, ls=ls[1])
    
    # weekly periodic effect
    cov_weekly = pm.gp.cov.Periodic(1, period=24*7, ls=ls[2])
    
    # altogether
    cov = cov_daily * cov_weekly + pm.gp.cov.WhiteNoise(1e-4)
    
    # sample hourly * daily alphas
    K = cov(np.arange(24 * 7)[:, None]).eval()
    mu = pm.Normal("mu", 0, 1, shape=len(K))
    alpha = pm.MvNormal("alpha", mu=mu, cov=K, shape=len(K))
    lmda = pm.Deterministic("lmda", np.exp(alpha))
    
    # Monthly multiplier
    nu = pm.Beta("nu", 12, 1.25, size=12)
    
    # sample from Gamma Poisson
    phi = pm.Exponential("phi", scale=3)
    
    c = pm.NegativeBinomial(
        "c", 
        alpha=phi, 
        mu=lmda[weekday * 24 + hour] * nu[month], 
        observed=np.array(filtered_data["count_outgoing"]), 
        dims="obs_id"
    )
    
    m_4.debug(verbose=True)
    # trace = pm.sample_prior_predictive(1000)
    trace = pm.sample(10000, tune=10000)
    trace.extend(pm.sample_posterior_predictive(trace))
    

## Trace results

In [ ]:
az.plot_forest(trace, var_names=["ls"])

In [ ]:
az.plot_forest(trace, var_names=["phi"])

In [ ]:
az.plot_forest(trace, var_names=["nu"])

In [ ]:
az.plot_trace(trace, compact=False, var_names=["ls", "nu", "phi", "lmda"]);
plt.tight_layout()

## Posterior vis

In [ ]:
m = plt.imshow(np.log(np.array(az.extract(trace.posterior)["lmda"]).mean(axis=1).reshape((7, 24))), cmap="inferno", interpolation=None)
plt.colorbar(m)

## Posterior predictions

In [ ]:
post_preds = np.array(az.extract(trace.posterior_predictive)["c"])
post_preds.shape

In [ ]:
day_post_preds = np.array([post_preds[i*24:(i+1)*24] for i in range(7)])
post_means = day_post_preds.mean(axis=2)
post_stds = day_post_preds.std(axis=2)

In [ ]:
hours = np.arange(24)

fig, axs = plt.subplots(nrows=7, sharex=True, sharey=True)
for i, (mean, std) in enumerate(zip(post_means, post_stds)):
    ax = axs[(i+5) % 7]
    ax.plot(hours, mean, label="Posterior mean", color="darkorange", linestyle="--")
    ax.fill_between(hours, (mean - std).clip(min=0), (mean + std), alpha=0.3, label="1 Std", color="darkorange")
    ax.fill_between(hours, (mean - 2*std).clip(min=0), (mean + 2*std), alpha=0.3, label="2 Std", color="darkorange")

plt.xticks(hours)
plt.xlim(0,23)
plt.ylim(-1,250)

fig.supxlabel("Hour of day")
fig.supylabel("Bikes counted (outgoing)")
fig.legend(loc="upper right")
plt.suptitle("Per-hour, per-day posterior predictive distribution of outgoing count to two std. (Thorndon Quay, 2022)")
plt.show()